In [69]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kendalltau
import pandas as pd
import os
import datetime

knmipath = '/nobackup/users/killaars/PhD_data/test'

flask_in = os.path.join(knmipath,'sample_coordinates_2007111000_2007111700.nc')
flask_mp = os.path.join(knmipath,'flask_output_TM5_MP_2007111000_2007111700.nc')
flask_esm = os.path.join(knmipath,'flask_output_ECE_2007111000_2007111700.nc')

flask_in_fh = nc.Dataset(flask_in,mode='r')
flask_mp_fh = nc.Dataset(flask_mp,mode='r')
flask_esm_fh = nc.Dataset(flask_esm,mode='r')

In [264]:
x = 40000000
fin_obs = flask_in_fh.variables['observed'][:x]
fin_date = flask_in_fh.variables['date_components'][:x]
obs_num = flask_in_fh.variables['obs_num'][:x]
lat = flask_in_fh.variables['latitude'][:x]
lon = flask_in_fh.variables['longitude'][:x]
alt = flask_in_fh.variables['altitude'][:x]
fmp_obs0 = flask_mp_fh.variables['flask'][:x,0] #Are 10 members. First check if it works with 1
fesm_obs = flask_esm_fh.variables['flask'][:x,0]

#check if the shapes are similar, meaning that they have the same number of points
print(np.shape(fin_obs),np.shape(fmp_obs0),np.shape(fesm_obs))

#Code to transform the date components of the input file to a pandas series of timestamps
N = (len(fin_obs))
base = datetime.datetime(1900, 1, 1)
time = np.array([base + datetime.timedelta(hours=i) for i in xrange(N)])
for i in range(N):
    dt = datetime.datetime(fin_date[i,0],fin_date[i,1],fin_date[i,2],fin_date[i,3],fin_date[i,4],fin_date[i,5])
    time[i] = pd.Timestamp(dt)

((25427,), (25427,), (25427,))
(25427,)


In [239]:
df = pd.DataFrame({'A_fin_obs' : fin_obs,
                   'B_fmp_obs' : fmp_obs0,
                   'C_fesm_obs': fesm_obs,
                   'D_alt'     : alt,
                   'E_lat'     : lat,
                   'F_lon'     : lon,
                   'G_date'    : time},index=obs_num)


PAL = (df.E_lat>67) & (df.E_lat<68) & (df.F_lon>24) & (df.F_lon<25) #PALLAS in Finland
CBW = (df.E_lat>51) & (df.E_lat<52) & (df.F_lon>4) & (df.F_lon<5) #Cabauw in the Netherlands
LEF = (df.E_lat>45) & (df.E_lat<46) & (df.F_lon>-91) & (df.F_lon<-90) #Park Falls in Wisconsin (USA)
JFJ = (df.E_lat>46) & (df.E_lat<47) & (df.F_lon>7) & (df.F_lon<8) #Jungfraujoch in Switserland
#df[JFJ]

In [236]:
fig, ax = plt.subplots()
mask = JFJ
ax.plot_date(x=df.G_date[mask], y=df.A_fin_obs[mask], color='g', alpha=.2, label='Observations')
ax.plot_date(x=df.G_date[mask], y=df.B_fmp_obs[mask], color='b', alpha=.2, label='TM5-Offline')
ax.plot_date(x=df.G_date[mask], y=df.C_fesm_obs[mask], color='r', alpha=.2, label='TM5-ESM')
plt.legend()
plt.show()


In [309]:
data = {'date': ['2014-05-01 18:47:05.069722', '2014-05-01 18:47:05.119994', '2014-05-02 18:47:05.178768', '2014-05-02 18:47:05.230071', '2014-05-02 18:47:05.230071', '2014-05-02 18:47:05.280592', '2014-05-03 18:47:05.332662', '2014-05-03 18:47:05.385109', '2014-05-04 18:47:05.436523', '2014-05-04 18:47:05.486877'],
        'deaths_regiment_1': [34, 43, 14, 15, 15, 14, 31, 25, 62, 41],
        'deaths_regiment_2': [52, 66, 78, 15, 15, 5, 25, 25, 86, 1],
        'deaths_regiment_3': [13, 73, 82, 58, 52, 87, 26, 5, 56, 75],
        'deaths_regiment_4': [44, 75, 26, 15, 15, 14, 54, 25, 24, 72],
        'deaths_regiment_5': [25, 24, 25, 15, 57, 68, 21, 27, 62, 5],
        'deaths_regiment_6': [84, 84, 26, 15, 15, 14, 26, 25, 62, 24],
        'deaths_regiment_7': [46, 57, 26, 15, 15, 14, 26, 25, 62, 41]}
df = pd.DataFrame(data, columns = ['date', 'battle_deaths', 'deaths_regiment_1', 'deaths_regiment_2',
                                   'deaths_regiment_3', 'deaths_regiment_4', 'deaths_regiment_5',
                                   'deaths_regiment_6', 'deaths_regiment_7'])
df = df.set_index(df.date)
sns.tsplot([df.deaths_regiment_1, df.deaths_regiment_2, df.deaths_regiment_3, df.deaths_regiment_4,
            df.deaths_regiment_5, df.deaths_regiment_6, df.deaths_regiment_7], color="indianred")
plt.show()

In [226]:
#1. TIME SERIES PLOT FOR SELECTED STATIONS
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import datetime

knmipath = '/nobackup/users/killaars/PhD_data/runs_may_2017/'
macbookpath = '/Users/killaars/PhD/test/'
path = knmipath

fin_obs = []
time = []
obs_num = []
lat = []
lon = []
alt = []
fmp_obs0 = []
fesm_obs = []
for root, dirs, files in os.walk(os.path.join(path,'sample_flasks')):
    for file in sorted(files):
        if file.startswith('sample_coordinates_'):
            #print(file,file[19:])
            #print 'flask_output_TM5_MP_'+file[19:]            
            #print 'flask_output_ECE_'+file[19:]
            TM5_MP_filename = 'flask_output_TM5_MP_'+file[19:]
            ECE_filename = 'flask_output_ECE_'+file[19:]
            
            flask_in = os.path.join(path,'sample_flasks',file)
            flask_mp = os.path.join(path,'TM5_offline',TM5_MP_filename)
            flask_esm = os.path.join(path,'TM5_ESM',ECE_filename)

            flask_in_fh = nc.Dataset(flask_in,mode='r')
            flask_mp_fh = nc.Dataset(flask_mp,mode='r')
            flask_esm_fh = nc.Dataset(flask_esm,mode='r')
            
            x = 40000000
            fin_obs = np.append(fin_obs,flask_in_fh.variables['observed'][:x])
            fin_date = flask_in_fh.variables['date_components'][:x]
            obs_num = np.append(obs_num,flask_in_fh.variables['obs_num'][:x])
            lat = np.append(lat,flask_in_fh.variables['latitude'][:x])
            lon = np.append(lon,flask_in_fh.variables['longitude'][:x])
            alt = np.append(alt,flask_in_fh.variables['altitude'][:x])
            fmp_obs0 = np.append(fmp_obs0,flask_mp_fh.variables['flask'][:x,0]) #Are 10 members. First check if it works with 1
            fesm_obs = np.append(fesm_obs,flask_esm_fh.variables['flask'][:x,0])
            
            flask_in_fh.close()
            flask_mp_fh.close()
            flask_esm_fh.close()

            #check if the shapes are similar, meaning that they have the same number of points
            #print(np.shape(fin_obs),np.shape(fmp_obs0),np.shape(fesm_obs))
            
            #Code to transform the date components of the input file to a pandas series of timestamps
            N = (len(fin_date))
            base = datetime.datetime(1900, 1, 1)
            time_intermediate = np.array([base + datetime.timedelta(hours=i) for i in range(N)])
            for i in range(N):
                dt = datetime.datetime(fin_date[i,0],fin_date[i,1],fin_date[i,2],fin_date[i,3],fin_date[i,4],fin_date[i,5])
                time_intermediate[i] = pd.Timestamp(dt)
            time = np.append(time,time_intermediate)    

df = pd.DataFrame({'A_fin_obs' : fin_obs*1000000,
                   'B_fmp_obs' : fmp_obs0*1000000,
                   'C_fesm_obs': fesm_obs*1000000,
                   'D_alt'     : alt,
                   'E_lat'     : lat,
                   'F_lon'     : lon,
                   'G_date'    : time},index=obs_num)


CBW = (df.E_lat>51) & (df.E_lat<52) & (df.F_lon>4) & (df.F_lon<5) & (df.D_alt<100) #Cabauw in the Netherlands
PAL = (df.E_lat>67) & (df.E_lat<68) & (df.F_lon>24) & (df.F_lon<25) #PALLAS in Finland
LEF = (df.E_lat>45) & (df.E_lat<46) & (df.F_lon>-91) & (df.F_lon<-90) #Park Falls in Wisconsin (USA)
JFJ = (df.E_lat>46) & (df.E_lat<47) & (df.F_lon>7) & (df.F_lon<8) #Jungfraujoch in Switserland
#print(df[JFJ])

f, (ax1, ax2,ax3,ax4) = plt.subplots(4, 1, sharex=True)
size = 2
alpha=.3
mask = LEF
ax1.set_title('LEF')
ax1.plot_date(x=df.G_date[mask], y=df.A_fin_obs[mask], color='g', alpha=alpha, ms=size, label='Observations')
ax1.plot_date(x=df.G_date[mask], y=df.B_fmp_obs[mask], color='b', alpha=alpha, ms=size, label='TM5-Offline')
ax1.plot_date(x=df.G_date[mask], y=df.C_fesm_obs[mask], color='r', alpha=alpha, ms=size, label='TM5-ESM')

mask=CBW
ax2.set_title('CBW')
ax2.plot_date(x=df.G_date[mask], y=df.A_fin_obs[mask], color='g', alpha=alpha, ms=size, label='Observations')
ax2.plot_date(x=df.G_date[mask], y=df.B_fmp_obs[mask], color='b', alpha=alpha, ms=size, label='TM5-Offline')
ax2.plot_date(x=df.G_date[mask], y=df.C_fesm_obs[mask], color='r', alpha=alpha, ms=size, label='TM5-ESM')

mask=PAL
ax3.set_title('PAL')
ax3.plot_date(x=df.G_date[mask], y=df.A_fin_obs[mask], color='g', alpha=alpha, ms=size, label='Observations')
ax3.plot_date(x=df.G_date[mask], y=df.B_fmp_obs[mask], color='b', alpha=alpha, ms=size, label='TM5-Offline')
ax3.plot_date(x=df.G_date[mask], y=df.C_fesm_obs[mask], color='r', alpha=alpha, ms=size, label='TM5-ESM')

mask=JFJ
ax4.set_title('JFJ')
ax4.plot_date(x=df.G_date[mask], y=df.A_fin_obs[mask], color='g', alpha=alpha, ms=size, label='Observations')
ax4.plot_date(x=df.G_date[mask], y=df.B_fmp_obs[mask], color='b', alpha=alpha, ms=size, label='TM5-Offline')
ax4.plot_date(x=df.G_date[mask], y=df.C_fesm_obs[mask], color='r', alpha=alpha, ms=size, label='TM5-ESM')

ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [231]:
#2. TIME SERIES PLOT FOR SELECTED STATIONS with errorbars - errorbars very small due to the very similar ensemble members
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import datetime

knmipath = '/nobackup/users/killaars/PhD_data/runs_may_2017'
macbookpath = '/Users/killaars/PhD/test/'
path = knmipath

fin_obs = []
time = []
obs_num = []
lat = []
lon = []
alt = []
fmp_obs0 = []
fmp_obs1 = []
fmp_obs2 = []
fmp_obs3 = []
fmp_obs4 = []
fmp_obs5 = []
fmp_obs6 = []
fmp_obs7 = []
fmp_obs8 = []
fmp_obs9 = []

fesm_obs0 = []
fesm_obs1 = []
fesm_obs2 = []
fesm_obs3 = []
fesm_obs4 = []
fesm_obs5 = []
fesm_obs6 = []
fesm_obs7 = []
fesm_obs8 = []
fesm_obs9 = []
for root, dirs, files in os.walk(os.path.join(path,'sample_flasks')):
    for file in sorted(files):
        if file.startswith('sample_coordinates_'):
            #print(file,file[19:])
            #print 'flask_output_TM5_MP_'+file[19:]            
            #print 'flask_output_ECE_'+file[19:]
            TM5_MP_filename = 'flask_output_TM5_MP_'+file[19:]
            ECE_filename = 'flask_output_ECE_'+file[19:]
            
            flask_in = os.path.join(path,'sample_flasks',file)
            flask_mp = os.path.join(path,'TM5_offline',TM5_MP_filename)
            flask_esm = os.path.join(path,'TM5_ESM',ECE_filename)

            flask_in_fh = nc.Dataset(flask_in,mode='r')
            flask_mp_fh = nc.Dataset(flask_mp,mode='r')
            flask_esm_fh = nc.Dataset(flask_esm,mode='r')
            
            x = 40000000
            fin_obs = np.append(fin_obs,flask_in_fh.variables['observed'][:x])
            fin_date = flask_in_fh.variables['date_components'][:x]
            obs_num = np.append(obs_num,flask_in_fh.variables['obs_num'][:x])
            lat = np.append(lat,flask_in_fh.variables['latitude'][:x])
            lon = np.append(lon,flask_in_fh.variables['longitude'][:x])
            alt = np.append(alt,flask_in_fh.variables['altitude'][:x])
            
            fmp_obs0 = np.append(fmp_obs0,flask_mp_fh.variables['flask'][:x,0]) #Are 10 members. First check if it works with 1
            fmp_obs1 = np.append(fmp_obs1,flask_mp_fh.variables['flask'][:x,1])
            fmp_obs2 = np.append(fmp_obs2,flask_mp_fh.variables['flask'][:x,2])
            fmp_obs3 = np.append(fmp_obs3,flask_mp_fh.variables['flask'][:x,3])
            fmp_obs4 = np.append(fmp_obs4,flask_mp_fh.variables['flask'][:x,4])
            fmp_obs5 = np.append(fmp_obs5,flask_mp_fh.variables['flask'][:x,5])
            fmp_obs6 = np.append(fmp_obs6,flask_mp_fh.variables['flask'][:x,6])
            fmp_obs7 = np.append(fmp_obs7,flask_mp_fh.variables['flask'][:x,7])
            fmp_obs8 = np.append(fmp_obs8,flask_mp_fh.variables['flask'][:x,8])
            fmp_obs9 = np.append(fmp_obs9,flask_mp_fh.variables['flask'][:x,9])
            
            fesm_obs0 = np.append(fesm_obs0,flask_esm_fh.variables['flask'][:x,0]) #Are 10 members. First check if it works with 1
            fesm_obs1 = np.append(fesm_obs1,flask_esm_fh.variables['flask'][:x,1])
            fesm_obs2 = np.append(fesm_obs2,flask_esm_fh.variables['flask'][:x,2])
            fesm_obs3 = np.append(fesm_obs3,flask_esm_fh.variables['flask'][:x,3])
            fesm_obs4 = np.append(fesm_obs4,flask_esm_fh.variables['flask'][:x,4])
            fesm_obs5 = np.append(fesm_obs5,flask_esm_fh.variables['flask'][:x,5])
            fesm_obs6 = np.append(fesm_obs6,flask_esm_fh.variables['flask'][:x,6])
            fesm_obs7 = np.append(fesm_obs7,flask_esm_fh.variables['flask'][:x,7])
            fesm_obs8 = np.append(fesm_obs8,flask_esm_fh.variables['flask'][:x,8])
            fesm_obs9 = np.append(fesm_obs9,flask_esm_fh.variables['flask'][:x,9])
            
            flask_in_fh.close()
            flask_mp_fh.close()
            flask_esm_fh.close()
            

            #check if the shapes are similar, meaning that they have the same number of points
            #print(np.shape(fin_obs),np.shape(fmp_obs0),np.shape(fesm_obs))
            
            #Code to transform the date components of the input file to a pandas series of timestamps
            N = (len(fin_date))
            base = datetime.datetime(1900, 1, 1)
            time_intermediate = np.array([base + datetime.timedelta(hours=i) for i in xrange(N)])
            for i in range(N):
                dt = datetime.datetime(fin_date[i,0],fin_date[i,1],fin_date[i,2],fin_date[i,3],fin_date[i,4],fin_date[i,5])
                time_intermediate[i] = pd.Timestamp(dt)
            time = np.append(time,time_intermediate)    

df = pd.DataFrame({'A_fin_obs' : fin_obs*1000000,
                   'B_fmp_obs' : fmp_obs0*1000000,
                   'C_fesm_obs': fesm_obs0*1000000,
                   'D_alt'     : alt,
                   'E_lat'     : lat,
                   'F_lon'     : lon,
                   'G_date'    : time},index=obs_num)

ensemble = pd.DataFrame({'A_fin_obs' : fin_obs*1000000,
                        'mp0' : fmp_obs0*1000000,
                        'mp1' : fmp_obs1*1000000,
                        'mp2' : fmp_obs2*1000000,
                        'mp3' : fmp_obs3*1000000,
                        'mp4' : fmp_obs4*1000000,
                        'mp5' : fmp_obs5*1000000,
                        'mp6' : fmp_obs6*1000000,
                        'mp7' : fmp_obs7*1000000,
                        'mp8' : fmp_obs8*1000000,
                        'mp9' : fmp_obs9*1000000,
                        'esm0' : fesm_obs0*1000000,
                        'esm1' : fesm_obs1*1000000,
                        'esm2' : fesm_obs2*1000000,
                        'esm3' : fesm_obs3*1000000,
                        'esm4' : fesm_obs4*1000000,
                        'esm5' : fesm_obs5*1000000,
                        'esm6' : fesm_obs6*1000000,
                        'esm7' : fesm_obs7*1000000,
                        'esm8' : fesm_obs8*1000000,
                        'esm9' : fesm_obs9*1000000,
                        'D_alt'     : alt,
                        'E_lat'     : lat,
                        'F_lon'     : lon,
                        'G_date'    : time},index=obs_num)
                  

CBW = (df.E_lat>51) & (df.E_lat<52) & (df.F_lon>4) & (df.F_lon<5) #Cabauw in the Netherlands
PAL = (df.E_lat>67) & (df.E_lat<68) & (df.F_lon>24) & (df.F_lon<25) #PALLAS in Finland
LEF = (df.E_lat>45) & (df.E_lat<46) & (df.F_lon>-91) & (df.F_lon<-90) #Park Falls in Wisconsin (USA)
JFJ = (df.E_lat>46) & (df.E_lat<47) & (df.F_lon>7) & (df.F_lon<8) #Jungfraujoch in Switserland

CBW = (ensemble.E_lat>51) & (ensemble.E_lat<52) & (ensemble.F_lon>4) & (ensemble.F_lon<5) & (ensemble.G_date>'2009-07-01')#Cabauw in the Netherlands
PAL = (ensemble.E_lat>67) & (ensemble.E_lat<68) & (ensemble.F_lon>24) & (ensemble.F_lon<25) #PALLAS in Finland
mask = CBW

ensemble = ensemble[mask]
#print(fmp.duplicated('G_date'))
#print(df[JFJ])
sns.tsplot(ensemble.A_fin_obs, color='g',alpha=.2)
sns.tsplot([ensemble.mp0,ensemble.mp1,ensemble.mp2,ensemble.mp3,ensemble.mp4,ensemble.mp5,
            ensemble.mp6,ensemble.mp7,ensemble.mp8,ensemble.mp9],color='b',alpha=.2)
sns.tsplot([ensemble.esm0,ensemble.esm1,ensemble.esm2,ensemble.esm3,ensemble.esm4,ensemble.esm5,
            ensemble.esm6,ensemble.esm7,ensemble.esm8,ensemble.esm9],color='r',alpha=.2)
plt.show()

In [244]:
#3. Altitude plots
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os,sys
import datetime

knmipath = '/nobackup/users/killaars/PhD_data/runs_may_2017'
macbookpath = '/Users/killaars/PhD/test/'
path = knmipath

fin_obs = []
time = []
obs_num = []
lat = []
lon = []
alt = []
fmp_obs0 = []
fesm_obs = []
for root, dirs, files in os.walk(os.path.join(path,'sample_flasks')):
    for file in sorted(files):
        if file.startswith('sample_coordinates_'):
            #print(file,file[19:])
            #print 'flask_output_TM5_MP_'+file[19:]            
            #print 'flask_output_ECE_'+file[19:]
            TM5_MP_filename = 'flask_output_TM5_MP_'+file[19:]
            ECE_filename = 'flask_output_ECE_'+file[19:]
            
            flask_in = os.path.join(path,'sample_flasks',file)
            flask_mp = os.path.join(path,'TM5_offline',TM5_MP_filename)
            flask_esm = os.path.join(path,'TM5_ESM',ECE_filename)

            flask_in_fh = nc.Dataset(flask_in,mode='r')
            flask_mp_fh = nc.Dataset(flask_mp,mode='r')
            flask_esm_fh = nc.Dataset(flask_esm,mode='r')
            
            x = 40000000
            fin_obs = np.append(fin_obs,flask_in_fh.variables['observed'][:x])
            fin_date = flask_in_fh.variables['date_components'][:x]
            obs_num = np.append(obs_num,flask_in_fh.variables['obs_num'][:x])
            lat = np.append(lat,flask_in_fh.variables['latitude'][:x])
            lon = np.append(lon,flask_in_fh.variables['longitude'][:x])
            alt = np.append(alt,flask_in_fh.variables['altitude'][:x])
            fmp_obs0 = np.append(fmp_obs0,flask_mp_fh.variables['flask'][:x,0]) #Are 10 members. First check if it works with 1
            fesm_obs = np.append(fesm_obs,flask_esm_fh.variables['flask'][:x,0])
            
            flask_in_fh.close()
            flask_mp_fh.close()
            flask_esm_fh.close()

            #check if the shapes are similar, meaning that they have the same number of points
            #print(np.shape(fin_obs),np.shape(fmp_obs0),np.shape(fesm_obs))
            
            #Code to transform the date components of the input file to a pandas series of timestamps
            N = (len(fin_date))
            base = datetime.datetime(1900, 1, 1)
            time_intermediate = np.array([base + datetime.timedelta(hours=i) for i in range(N)])
            for i in range(N):
                dt = datetime.datetime(fin_date[i,0],fin_date[i,1],fin_date[i,2],fin_date[i,3],fin_date[i,4],fin_date[i,5])
                time_intermediate[i] = pd.Timestamp(dt)
            time = np.append(time,time_intermediate)    

df = pd.DataFrame({'A_fin_obs' : fin_obs*1000000,
                   'B_fmp_obs' : fmp_obs0*1000000,
                   'C_fesm_obs': fesm_obs*1000000,
                   'D_alt'     : alt,
                   'E_lat'     : lat,
                   'F_lon'     : lon,
                   'G_date'    : time},index=obs_num)


CBW = (df.E_lat>51) & (df.E_lat<52) & (df.F_lon>4) & (df.F_lon<5) #Cabauw in the Netherlands
PAL = (df.E_lat>67) & (df.E_lat<68) & (df.F_lon>24) & (df.F_lon<25) #PALLAS in Finland
LEF = (df.E_lat>45) & (df.E_lat<46) & (df.F_lon>-91) & (df.F_lon<-90) #Park Falls in Wisconsin (USA)
JFJ = (df.E_lat>46) & (df.E_lat<47) & (df.F_lon>7) & (df.F_lon<8) #Jungfraujoch in Switserland
GLOB= (df.E_lat>-200) & (df.E_lat<200) & (df.F_lon>-200) & (df.F_lon<200) #Global mask to view all the data

POL = (df.E_lat>80) & (df.E_lat<100) | (df.E_lat<-80) & (df.E_lat>=-100)   # Poles
SUBPOL = (df.E_lat>60) & (df.E_lat<=80) | (df.E_lat<-60) & (df.E_lat>=-80) # Subpolair areas
TEM = (df.E_lat>40) & (df.E_lat<=60) | (df.E_lat<-40) & (df.E_lat>=-60)    # Temperate zones
SUBTRO = (df.E_lat>20) & (df.E_lat<=40) | (df.E_lat<-20) & (df.E_lat>=-40) # Subtropics
TRO = (df.E_lat>-20) & (df.E_lat<=20)                                      # Tropics

HIGH = (df.C_fesm_obs>500) 
print(df[HIGH].head)

f, (ax1, ax2,ax3,ax4,ax5) = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(20, 10))
size = 2
alpha=.3
mask = POL

# df['A_fin_obs'] = df['A_fin_obs']*1000000
# df['B_fmp_obs'] = df['B_fmp_obs']*1000000
# df['C_fesm_obs'] = df['C_fesm_obs']*1000000


sys.exit()


#Groups A_fin_obs by the altitude. The altitude is aggregated for each 50m and several statistics are done on each altitude value
alt_fin_obs = df[mask]['A_fin_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
#standard error of the mean is calculated by dividing the standard deviation by the square root of the number of measurements
alt_fin_obs['error']= alt_fin_obs['std']/np.sqrt(alt_fin_obs['count'])

alt_fmp_obs = df[mask]['B_fmp_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fmp_obs['error']= alt_fmp_obs['std']/np.sqrt(alt_fmp_obs['count'])

alt_fesm_obs = df[mask]['C_fesm_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fesm_obs['error']= alt_fesm_obs['std']/np.sqrt(alt_fesm_obs['count'])
alt = np.arange(25,df[mask]['D_alt'].max()+25, 50)
ax1.errorbar(alt_fin_obs['mean'], alt,xerr=alt_fin_obs['error'], fmt='o', color = 'g', alpha=alpha, ms=size,label='Observations')
ax1.errorbar(alt_fmp_obs['mean'], alt,xerr=alt_fmp_obs['error'], fmt='o', color = 'b', alpha=alpha, ms=size, label='TM5-Offline')
ax1.errorbar(alt_fesm_obs['mean'], alt,xerr=alt_fesm_obs['error'], fmt='o', color = 'r', alpha=alpha, ms=size, label='TM5-ESM')
ax1.set_title("Poles")
#plt.xlim(380,420)
plt.ylim(0,max(df.D_alt))

mask = SUBPOL
#Groups A_fin_obs by the altitude. The altitude is aggregated for each 50m and several statistics are done on each altitude value
alt_fin_obs = df[mask]['A_fin_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
#standard error of the mean is calculated by dividing the standard deviation by the square root of the number of measurements
alt_fin_obs['error']= alt_fin_obs['std']/np.sqrt(alt_fin_obs['count'])

alt_fmp_obs = df[mask]['B_fmp_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fmp_obs['error']= alt_fmp_obs['std']/np.sqrt(alt_fmp_obs['count'])

alt_fesm_obs = df[mask]['C_fesm_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fesm_obs['error']= alt_fesm_obs['std']/np.sqrt(alt_fesm_obs['count'])
alt = np.arange(25,df[mask]['D_alt'].max()+25, 50)
ax2.errorbar(alt_fin_obs['mean'], alt,xerr=alt_fin_obs['error'], fmt='o', color = 'g', alpha=alpha, ms=size,label='Observations')
ax2.errorbar(alt_fmp_obs['mean'], alt,xerr=alt_fmp_obs['error'], fmt='o', color = 'b', alpha=alpha, ms=size, label='TM5-Offline')
ax2.errorbar(alt_fesm_obs['mean'], alt,xerr=alt_fesm_obs['error'], fmt='o', color = 'r', alpha=alpha, ms=size, label='TM5-ESM')
ax2.set_title("Sub Polair")
mask = TEM
#Groups A_fin_obs by the altitude. The altitude is aggregated for each 50m and several statistics are done on each altitude value
alt_fin_obs = df[mask]['A_fin_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
#standard error of the mean is calculated by dividing the standard deviation by the square root of the number of measurements
alt_fin_obs['error']= alt_fin_obs['std']/np.sqrt(alt_fin_obs['count'])

alt_fmp_obs = df[mask]['B_fmp_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fmp_obs['error']= alt_fmp_obs['std']/np.sqrt(alt_fmp_obs['count'])

alt_fesm_obs = df[mask]['C_fesm_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fesm_obs['error']= alt_fesm_obs['std']/np.sqrt(alt_fesm_obs['count'])
alt = np.arange(25,df[mask]['D_alt'].max()+25, 50)
ax3.errorbar(alt_fin_obs['mean'], alt,xerr=alt_fin_obs['error'], fmt='o', color = 'g', alpha=alpha, ms=size,label='Observations')
ax3.errorbar(alt_fmp_obs['mean'], alt,xerr=alt_fmp_obs['error'], fmt='o', color = 'b', alpha=alpha, ms=size, label='TM5-Offline')
ax3.errorbar(alt_fesm_obs['mean'], alt,xerr=alt_fesm_obs['error'], fmt='o', color = 'r', alpha=alpha, ms=size, label='TM5-ESM')
ax3.set_title("Temperate")
mask = SUBTRO
#Groups A_fin_obs by the altitude. The altitude is aggregated for each 50m and several statistics are done on each altitude value
alt_fin_obs = df[mask]['A_fin_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
#standard error of the mean is calculated by dividing the standard deviation by the square root of the number of measurements
alt_fin_obs['error']= alt_fin_obs['std']/np.sqrt(alt_fin_obs['count'])

alt_fmp_obs = df[mask]['B_fmp_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fmp_obs['error']= alt_fmp_obs['std']/np.sqrt(alt_fmp_obs['count'])

alt_fesm_obs = df[mask]['C_fesm_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fesm_obs['error']= alt_fesm_obs['std']/np.sqrt(alt_fesm_obs['count'])
alt = np.arange(25,df[mask]['D_alt'].max()+25, 50)
ax4.errorbar(alt_fin_obs['mean'], alt,xerr=alt_fin_obs['error'], fmt='o', color = 'g', alpha=alpha, ms=size,label='Observations')
ax4.errorbar(alt_fmp_obs['mean'], alt,xerr=alt_fmp_obs['error'], fmt='o', color = 'b', alpha=alpha, ms=size, label='TM5-Offline')
ax4.errorbar(alt_fesm_obs['mean'], alt,xerr=alt_fesm_obs['error'], fmt='o', color = 'r', alpha=alpha, ms=size, label='TM5-ESM')
ax4.set_title("Subtropics")
mask = TRO
#Groups A_fin_obs by the altitude. The altitude is aggregated for each 50m and several statistics are done on each altitude value
alt_fin_obs = df[mask]['A_fin_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
#standard error of the mean is calculated by dividing the standard deviation by the square root of the number of measurements
alt_fin_obs['error']= alt_fin_obs['std']/np.sqrt(alt_fin_obs['count'])

alt_fmp_obs = df[mask]['B_fmp_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fmp_obs['error']= alt_fmp_obs['std']/np.sqrt(alt_fmp_obs['count'])

alt_fesm_obs = df[mask]['C_fesm_obs'].groupby(pd.cut(df[mask]['D_alt'], np.arange(0, df[mask]['D_alt'].max()+50, 50))).agg({'mean':np.mean,
                                                                                                 'count':"count",
                                                                                                 'std':np.std})
alt_fesm_obs['error']= alt_fesm_obs['std']/np.sqrt(alt_fesm_obs['count'])
alt = np.arange(25,df[mask]['D_alt'].max()+25, 50)
ax5.errorbar(alt_fin_obs['mean'], alt,xerr=alt_fin_obs['error'], fmt='o', color = 'g', alpha=alpha, ms=size,label='Observations')
ax5.errorbar(alt_fmp_obs['mean'], alt,xerr=alt_fmp_obs['error'], fmt='o', color = 'b', alpha=alpha, ms=size, label='TM5-Offline')
ax5.errorbar(alt_fesm_obs['mean'], alt,xerr=alt_fesm_obs['error'], fmt='o', color = 'r', alpha=alpha, ms=size, label='TM5-ESM')
ax5.set_title("Tropics")

ax1.set(ylabel='Altitude (m)')
ax3.set(xlabel='CO2 concentration (ppm)')
plt.tight_layout()
plt.show()

<bound method DataFrame.head of           A_fin_obs   B_fmp_obs  C_fesm_obs  D_alt      E_lat    F_lon  \
24908.0  409.310014  420.544267  505.697331  990.0  47.801102  11.0245   

                     G_date  
24908.0 2009-01-07 11:55:00  >


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [228]:
#4. latitude heatmap
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import datetime

knmipath = '/nobackup/users/killaars/PhD_data/runs_may_2017'
macbookpath = '/Users/killaars/PhD/test/'
path = knmipath

fin_obs = []
time = []
obs_num = []
lat = []
lon = []
alt = []
fmp_obs0 = []
fesm_obs = []
for root, dirs, files in os.walk(os.path.join(path,'sample_flasks')):
    for file in sorted(files):
        if file.startswith('sample_coordinates_'):
            #print(file,file[19:])
            #print 'flask_output_TM5_MP_'+file[19:]            
            #print 'flask_output_ECE_'+file[19:]
            TM5_MP_filename = 'flask_output_TM5_MP_'+file[19:]
            ECE_filename = 'flask_output_ECE_'+file[19:]
            
            flask_in = os.path.join(path,'sample_flasks',file)
            flask_mp = os.path.join(path,'TM5_offline',TM5_MP_filename)
            flask_esm = os.path.join(path,'TM5_ESM',ECE_filename)

            flask_in_fh = nc.Dataset(flask_in,mode='r')
            flask_mp_fh = nc.Dataset(flask_mp,mode='r')
            flask_esm_fh = nc.Dataset(flask_esm,mode='r')
            
            x = 40000000
            fin_obs = np.append(fin_obs,flask_in_fh.variables['observed'][:x])
            fin_date = flask_in_fh.variables['date_components'][:x]
            obs_num = np.append(obs_num,flask_in_fh.variables['obs_num'][:x])
            lat = np.append(lat,flask_in_fh.variables['latitude'][:x])
            lon = np.append(lon,flask_in_fh.variables['longitude'][:x])
            alt = np.append(alt,flask_in_fh.variables['altitude'][:x])
            fmp_obs0 = np.append(fmp_obs0,flask_mp_fh.variables['flask'][:x,0]) #Are 10 members. First check if it works with 1
            fesm_obs = np.append(fesm_obs,flask_esm_fh.variables['flask'][:x,0])
            
            flask_in_fh.close()
            flask_mp_fh.close()
            flask_esm_fh.close()

            #check if the shapes are similar, meaning that they have the same number of points
            #print(np.shape(fin_obs),np.shape(fmp_obs0),np.shape(fesm_obs))
            
            #Code to transform the date components of the input file to a pandas series of timestamps
            N = (len(fin_date))
            base = datetime.datetime(1900, 1, 1)
            time_intermediate = np.array([base + datetime.timedelta(hours=i) for i in range(N)])
            for i in range(N):
                dt = datetime.datetime(fin_date[i,0],fin_date[i,1],fin_date[i,2],fin_date[i,3],fin_date[i,4],fin_date[i,5])
                time_intermediate[i] = pd.Timestamp(dt)
            time = np.append(time,time_intermediate)    

df = pd.DataFrame({'A_fin_obs' : fin_obs*1000000,
                   'B_fmp_obs' : fmp_obs0*1000000,
                   'C_fesm_obs': fesm_obs*1000000,
                   'D_alt'     : alt,
                   'E_lat'     : lat,
                   'F_lon'     : lon,
                   'G_date'    : time},index=obs_num)

GLOB= (df.E_lat>-200) & (df.E_lat<200) & (df.F_lon>-200) & (df.F_lon<200) #Global mask to view all the data
mask=GLOB

# lat_fin_obs = df[mask]['A_fin_obs'].groupby([pd.cut(df[mask]['E_lat'],np.arange(-90,100,10)),pd.cut(df[mask]['D_alt'],np.arange(0,13000,1000))]).agg({'mean':np.mean,
#                                                                                                  'count':"count",
#                                                                                                  'std':np.std})
# Calculate the mean of all the values that fit in the numpy array boxes and use the unstack function to get rid of the multi-index dataframe
lat_fin_obs_mean = df[mask]['A_fin_obs'].groupby([pd.cut(df[mask]['D_alt'],np.arange(0,14000,1000)),
                                                  pd.cut(df[mask]['E_lat'],np.arange(-90,100,20))]).agg({'mean':np.mean}).unstack()
lat_fmp_obs_mean = df[mask]['B_fmp_obs'].groupby([pd.cut(df[mask]['D_alt'],np.arange(0,14000,1000)),
                                                  pd.cut(df[mask]['E_lat'],np.arange(-90,100,20))]).agg({'mean':np.mean}).unstack()
lat_fesm_obs_mean = df[mask]['C_fesm_obs'].groupby([pd.cut(df[mask]['D_alt'],np.arange(0,14000,1000)),
                                                    pd.cut(df[mask]['E_lat'],np.arange(-90,100,20))]).agg({'mean':np.mean}).unstack()

# Do the same for the min and max, subtract them and unstack the resulting difference column.
lat_fin_obs_diff = df[mask]['A_fin_obs'].groupby([pd.cut(df[mask]['D_alt'],np.arange(0,14000,1000)),
                                                  pd.cut(df[mask]['E_lat'],np.arange(-90,100,20))]).agg({'max':np.max,'min':np.min})
lat_fin_obs_diff['diff']=lat_fin_obs_diff['max']-lat_fin_obs_diff['min']
lat_fin_obs_diff = lat_fin_obs_diff['diff'].unstack()

# invert the altitude axis
lat_fin_obs_mean = lat_fin_obs_mean.reindex(index=lat_fin_obs_mean.index[::-1])
lat_fmp_obs_mean = lat_fmp_obs_mean.reindex(index=lat_fmp_obs_mean.index[::-1])
lat_fesm_obs_mean = lat_fesm_obs_mean.reindex(index=lat_fesm_obs_mean.index[::-1])

lat_fin_obs_diff = lat_fin_obs_diff.reindex(index=lat_fin_obs_diff.index[::-1])

alt_index = np.arange(12500,-500,-1000)
lat_index = np.arange(-80,90,20)

f, (ax1) = plt.subplots(1, 1)
ax1 = sns.heatmap(lat_fin_obs_mean,xticklabels=lat_index,yticklabels=alt_index, linewidths=.5, cmap="YlGnBu")    
ax1.set(xlabel='Latitude', ylabel='Altitude (m)',title='Observed CO2 concentrations in ppm')
for item in (ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(8)
plt.show()
# ax2 = sns.heatmap(lat_fmp_obs-lat_fin_obs,xticklabels=lat_index,yticklabels=alt_index, linewidths=.5, cmap="YlGnBu")    
# ax2.set(xlabel='Latitude', ylabel='Altitude (m)',title='TM5-Offline CO2 concentrations in ppm')
    
# ax3 = sns.heatmap(lat_fesm_obs-lat_fin_obs,xticklabels=lat_index,yticklabels=alt_index, linewidths=.5, cmap="YlGnBu")    
# ax3.set(xlabel='Latitude', ylabel='Altitude (m)',title='TM5-ECE CO2 concentrations in ppm')

# plt.show()


In [227]:
#5. amplitude - latitude plot
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import datetime

knmipath = '/nobackup/users/killaars/PhD_data/runs_may_2017'
macbookpath = '/Users/killaars/PhD/test/'
path = knmipath

fin_obs = []
time = []
obs_num = []
lat = []
lon = []
alt = []
fmp_obs0 = []
fesm_obs = []
for root, dirs, files in os.walk(os.path.join(path,'sample_flasks')):
    for file in sorted(files):
        if file.startswith('sample_coordinates_'):
            #print(file,file[19:])
            #print 'flask_output_TM5_MP_'+file[19:]            
            #print 'flask_output_ECE_'+file[19:]
            TM5_MP_filename = 'flask_output_TM5_MP_'+file[19:]
            ECE_filename = 'flask_output_ECE_'+file[19:]
            
            flask_in = os.path.join(path,'sample_flasks',file)
            flask_mp = os.path.join(path,'TM5_offline',TM5_MP_filename)
            flask_esm = os.path.join(path,'TM5_ESM',ECE_filename)

            flask_in_fh = nc.Dataset(flask_in,mode='r')
            flask_mp_fh = nc.Dataset(flask_mp,mode='r')
            flask_esm_fh = nc.Dataset(flask_esm,mode='r')
            
            x = 40000000
            fin_obs = np.append(fin_obs,flask_in_fh.variables['observed'][:x])
            fin_date = flask_in_fh.variables['date_components'][:x]
            obs_num = np.append(obs_num,flask_in_fh.variables['obs_num'][:x])
            lat = np.append(lat,flask_in_fh.variables['latitude'][:x])
            lon = np.append(lon,flask_in_fh.variables['longitude'][:x])
            alt = np.append(alt,flask_in_fh.variables['altitude'][:x])
            fmp_obs0 = np.append(fmp_obs0,flask_mp_fh.variables['flask'][:x,0]) #Are 10 members. First check if it works with 1
            fesm_obs = np.append(fesm_obs,flask_esm_fh.variables['flask'][:x,0])
            
            flask_in_fh.close()
            flask_mp_fh.close()
            flask_esm_fh.close()

            #check if the shapes are similar, meaning that they have the same number of points
            #print(np.shape(fin_obs),np.shape(fmp_obs0),np.shape(fesm_obs))
            
            #Code to transform the date components of the input file to a pandas series of timestamps
            N = (len(fin_date))
            base = datetime.datetime(1900, 1, 1)
            time_intermediate = np.array([base + datetime.timedelta(hours=i) for i in range(N)])
            for i in range(N):
                dt = datetime.datetime(fin_date[i,0],fin_date[i,1],fin_date[i,2],fin_date[i,3],fin_date[i,4],fin_date[i,5])
                time_intermediate[i] = pd.Timestamp(dt)
            time = np.append(time,time_intermediate)    

df = pd.DataFrame({'A_fin_obs' : fin_obs*1000000,
                   'B_fmp_obs' : fmp_obs0*1000000,
                   'C_fesm_obs': fesm_obs*1000000,
                   'D_alt'     : alt,
                   'E_lat'     : lat,
                   'F_lon'     : lon,
                   'G_date'    : time},index=obs_num)

MLO = (df.E_lat>19) & (df.E_lat<20) & (df.F_lon>-155) & (df.F_lon<-154) #Mauna Loa, Hawai, US
ALT = (df.E_lat>82) & (df.E_lat<83) & (df.F_lon>-63) & (df.F_lon<-62) #Alert, Alaska, US
ASC = (df.E_lat>-8) & (df.E_lat<-7) & (df.F_lon>-15) & (df.F_lon<-14) #Ascencion Island (Middle Atlantic)
AMT = (df.E_lat>45) & (df.E_lat<46) & (df.F_lon>-69) & (df.F_lon<-68) #Argyle, Maine, US
CPT = (df.E_lat>-36) & (df.E_lat<-35) & (df.F_lon>18) & (df.F_lon<19) #Cape Town, South Africa
GLOB= (df.E_lat>-200) & (df.E_lat<200) & (df.F_lon>-200) & (df.F_lon<200) #Global mask to view all the data
masks = [MLO,ALT,ASC,AMT,CPT]

#For stations: empty arrays, filled with the difference between the min and the max value at each location. The 'lat' array
#consists of the latitude of the station
fin_diff = []
fmp_diff = []
fesm_diff = []
lat = []
for mask in masks:
    fin_diff=np.append(fin_diff,(df[mask]['A_fin_obs'].max()-df[mask]['A_fin_obs'].min()))
    fmp_diff=np.append(fmp_diff,(df[mask]['B_fmp_obs'].max()-df[mask]['B_fmp_obs'].min()))
    fesm_diff=np.append(fesm_diff,(df[mask]['C_fesm_obs'].max()-df[mask]['C_fesm_obs'].min()))
    lat =  np.append(lat,df[mask]['E_lat'].max())

                                                                                               
# Calculate some statistics of all the values that fit in the numpy array boxes and calculate the amplitude of the timeseries
lat_fin_obs = df['A_fin_obs'].groupby(pd.cut(df['E_lat'],np.arange(-90,95,5))).agg({'mean':np.mean,'min':np.min,'max':np.max})
lat_fin_obs['diff'] = lat_fin_obs['max']-lat_fin_obs['min']

lat_fmp_obs = df['B_fmp_obs'].groupby(pd.cut(df['E_lat'],np.arange(-90,95,5))).agg({'mean':np.mean,'min':np.min,'max':np.max})
lat_fmp_obs['diff'] = lat_fmp_obs['max']-lat_fmp_obs['min']

lat_fesm_obs = df['C_fesm_obs'].groupby(pd.cut(df['E_lat'],np.arange(-90,95,5))).agg({'mean':np.mean,'min':np.min,'max':np.max})
lat_fesm_obs['diff'] = lat_fesm_obs['max']-lat_fesm_obs['min']

f, (ax1,ax2) = plt.subplots(1, 2,sharey=True,sharex=True)
plt.suptitle('Observed CO2 concentrations in ppm')
size = 2
alpha=.3

ax1.scatter(lat,fin_diff,color='g', label='Observations')
ax1.scatter(lat,fmp_diff,color='b', label='TM5-MP')
ax1.scatter(lat,fesm_diff,color='r', label='TM5-ECE')
ax2.scatter(np.arange(-90,90,5),lat_fin_obs['diff'],color='g', label='Observations')
ax2.scatter(np.arange(-90,90,5),lat_fmp_obs['diff'],color='b', label='TM5-MP')
ax2.scatter(np.arange(-90,90,5),lat_fesm_obs['diff'],color='r', label='TM5-ECE')
ax1.set(xlabel='Latitude', ylabel='Concentration CO2 (ppm)')
ax2.set(xlabel='Latitude')

plt.ylim(np.min(lat_fin_obs['diff']),np.max(lat_fin_obs['diff']))
plt.xlim(-90,90)
plt.xticks(np.arange(-90, 90+30, 30))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()